<a href="https://colab.research.google.com/github/MehmedJahanBakhsh/Streaming-Analytics-Project/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download Dataset

In [7]:
 ! pip install -q kaggle

In [8]:
from google.colab import files

In [9]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"mohammadjahanbakhsh7","key":"06c125db1cdc8b89ffe89766e3ab9d7a"}'}

In [10]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
cp kaggle.json ~/.kaggle/

In [12]:
! chmod 600 ~/.kaggle/kaggle.json

In [13]:
! kaggle datasets download 'eliasdabbas/web-server-access-logs'

web-server-access-logs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [15]:
! unzip web-server-access-logs.zip -d dataset

Archive:  web-server-access-logs.zip
replace dataset/access.log? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: dataset/access.log      
replace dataset/client_hostname.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Parsing logs and plotting load

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
# parse data to DataFrame

data_arr = []
with open('/content/dataset/access.log') as log:
    for line in log.readlines():
        match = re.match(r'(.+) - - \[(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2}) \+\d{4}\] \"(\w+) (.*?) .+', line)
        if match:
            data_arr.append([match.group(1), match.group(2), match.group(3), match.group(4)])

data = pd.DataFrame(data_arr, columns=['ip_address', 'datetime', 'request_type', 'path'])

In [ ]:
#Convert string datetime to Timestamp

data['datetime'] = data.datetime.apply(lambda x: x.replace('Jan', '01'))
data['datetime'] = pd.to_datetime(data.datetime, format='%d/%m/%Y:%H:%M:%S')

In [ ]:
#add datetime columns to group by

data['day'] = data['datetime'].apply(lambda x: x.day)
data['hour'] = data['datetime'].apply(lambda x: x.hour)
data['minute'] = data['datetime'].apply(lambda x: x.minute)

In [ ]:
#aggregate by minute and by hour

aggByMin = data.groupby(['day', 'hour', 'minute'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type'], axis=1)

aggByHour = data.groupby(['day', 'hour'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type', 'minute', 'path'], axis=1)

aggByDay = data.groupby(['day'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type', 'hour', 'minute', 'path'], axis=1)

In [ ]:
#plot it

plt.plot(aggByMin['count'], label='load by minute')
plt.legend()

In [ ]:
plt.plot(aggByHour['count'], label='load by hour')
plt.legend()

In [ ]:
plt.plot(aggByDay['count'], label='load by day')
plt.legend()

ARIMA

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pandas.plotting import register_matplotlib_converters
import statsmodels.api as sm
register_matplotlib_converters()

In [ ]:
model = sm.tsa.arima.ARIMA(data, order=(1,1,1))
results = model.fit()
plt.plot(data)
plt.plot(results.fittedvalues, color='red')

In [ ]:
results.summary()

In [ ]:
predictions_ARIMA_diff = pd.Series(results.fittedvalues, copy=True)
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
predictions_ARIMA_log = pd.Series(df_log['requests'].iloc[0], index=df_log.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(df)
plt.plot(predictions_ARIMA)

In [ ]:
plot_predict(results)

In [ ]:
df['forecast']=results.predict()
df[['requests','forecast']].plot(figsize=(12,8))